<a href="https://colab.research.google.com/github/TurkuNLP/intro-to-nlp/blob/master/mlp_imdb_hf_dset_and_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Before we start running our own Python code, install the required Python packages using [pip](https://en.wikipedia.org/wiki/Pip):

* [`transformers`](https://huggingface.co/docs/transformers/index) is a popular deep learning package primarily on top of torch, we need to reinstall it with the [torch] configuration (might take a substantial amount of time)
* [`datasets`](https://huggingface.co/docs/datasets/) provides support for loading, creating, and manipulating datasets
* evaluate is a library of performance metrics (like accuracy etc)

**You will likely need to do a Runtime/Restart session for everything to work after the installation.**

In [1]:
# !pip3 install -q datasets evaluate
# !pip install transformers[torch]

(Above, the `!` at the start of the line tells the notebook to run the line as an operating system command rather than Python code, and the `-q` argument to `pip` runs the command in "quiet" mode, with less output.)

---

# Get and prepare data

*   Let us work with the IMDB dataset of movie review sentiment
*   25,000 positive reviews
*   25,000 negative reviews
*   50,000 unlabeled reviews (which we discard for the time being)


In [2]:
from pprint import pprint #pprint => pretty-print, I use it occassionally throughout the notebook
import datasets
import torch
dset=datasets.load_dataset("imdb")
pprint(dset)

c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
dset=dset.shuffle() #This is never a bad idea, datasets may have ordering to them, which is not what we want
del dset["unsupervised"] #Delete the unlabeled part of the dataset, we don't need it for anything

In [4]:
pprint(dset['train'][0]['text'])
print(dset['train'][0]['label'])

('There were a lot of films made by Hollywood during the war years that were '
 'designed to drum up support for our troops from the public. Seen today, some '
 'might dismiss them or just see them as propaganda--which they technically '
 'are, but of a positive sort and meant to unify the nation. This film is a '
 'pretty effective and entertaining example of the genre--having a pretty '
 "realistic script and good production values. Pat O'Brien plays pretty much "
 'the same character he played in MANY other films (you know, the '
 'tough-talking, hard-driven but "swell guy"). Randolph Scott is, as always, '
 'competent and entertaining and the rest of the extras are excellent (look '
 'for a young Robert Ryan as one of the bombardiers in training). While the '
 'story is reminiscent of several other movies about our pilots and crews, the '
 'film is well-crafted enough to make it interesting and not too far-fetched. '
 'That it, perhaps, except for the very end--where the film is a 

## Tokenize and map vocabulary
         
*   We need to achieve two complementary tasks
*   **Tokenize** split the text into units which can be interpreted as features (words in this case)
*   **Map vocabulary** build the feature vector for each example
*   Since this is NLP, here it means listing the non-zero elements of the feature vector, or in other words the indices of the vocabulary items
* Since we work with the bag of words (BoW) representation, these do not need to be (and are not) in the order in which they appear in the text
* These indices then refer to the rows in the embedding matrix
*   A traditional and well-tested way it to use sklearn's feature extraction package
*   CountVectorizer is most likely what we want in here, because we only want the ids, nothing else
* But for other NLP work the TfidfVectorizer is also very handy



In [5]:
import sklearn.feature_extraction

# max_features means the size of the vocabulary
# which means max_features most-common words
vectorizer=sklearn.feature_extraction.text.CountVectorizer(binary=True,max_features=20000)

texts=[ex["text"] for ex in dset["train"]] #get a list of all texts from the training data
vectorizer.fit(texts) #"Trains" the vectorizer, i.e. builds its vocabulary


CountVectorizer(binary=True, max_features=20000)

# Building the feature vectors

* This is super-easy with the vectorizer
* It produces a sparse matrix of the non-zero elements

In [6]:
def vectorize_example(ex):
    vectorized=vectorizer.transform([ex["text"]]) # [...] because the vectorizer expects a list/iterable over inputs, not one input
    non_zero_features=vectorized.nonzero()[1] #.nonzero gives a pair of (rows,columns), we want the columns
    non_zero_features+=1 #feature index 0 will have a special meaning
                         # so let us not produce it by adding +1 to everything
    return {"input_ids":non_zero_features}

vectorized=vectorize_example(dset["train"][0])

In [7]:
print(vectorized)

{'input_ids': array([  309,   663,   727,   774,   793,   887,  1115,  1207,  1980,
        2248,  2390,  2604,  2625,  3053,  3696,  4208,  4293,  4951,
        5265,  5590,  5617,  5697,  5835,  6044,  6118,  6141,  6357,
        6367,  6370,  6543,  6663,  6815,  6878,  6887,  7120,  7127,
        7346,  7593,  7801,  8103,  8241,  8325,  8350,  8599,  8649,
        9085,  9391,  9428,  9602,  9630,  9890, 10090, 10475, 10638,
       10681, 10703, 10829, 10886, 10970, 11176, 11190, 11376, 11723,
       11765, 11778, 11942, 12005, 12202, 12214, 12269, 12363, 12439,
       12445, 12504, 12564, 12565, 12574, 12627, 12905, 13059, 13238,
       13325, 13333, 13512, 13713, 13778, 13805, 13879, 13982, 14231,
       14340, 14641, 14825, 15085, 15297, 15370, 15429, 15567, 15603,
       15682, 15696, 15791, 15830, 16354, 16542, 16554, 16592, 17039,
       17075, 17404, 17532, 17648, 17727, 17752, 17893, 17897, 17910,
       17929, 17944, 17968, 18115, 18122, 18166, 18175, 18221, 18285,
      

In [8]:
# We can map back to vocabulary and check that everything works
# vectorizer.vocabulary_ is a dictionary {key:word, value:idx}

idx2word=dict((i,w) for (w,i) in vectorizer.vocabulary_.items()) #inverse the vocab dictionary
words=[]
for idx in vectorized["input_ids"]:
    words.append(idx2word[idx-1]) ## It is easy to forgot we moved all by +1
pprint(", ".join(words)) #This is now the bag of words representation of the document

('about, airplane, all, also, always, and, are, as, bit, both, brien, but, by, '
 'character, competent, crafted, crews, designed, dismiss, driven, drum, '
 'during, effective, end, enough, entertaining, example, excellent, except, '
 'extras, far, fetched, film, films, footage, for, from, genre, good, guy, '
 'hard, having, he, history, hollywood, in, integrated, interesting, is, it, '
 'just, know, like, look, lot, lovers, made, make, many, me, meant, might, '
 'mostly, movies, much, nation, negative, not, notice, nuts, of, one, only, '
 'or, other, others, our, over, pat, perhaps, pilots, played, plays, positive, '
 'pretty, probably, production, propaganda, public, randolph, realistic, '
 'reminiscent, rest, robert, ryan, same, satisfying, scott, script, see, seen, '
 'serious, several, sloppily, some, somewhat, sort, stock, story, support, '
 'swell, talking, teachers, technically, that, the, them, there, they, this, '
 'to, today, too, top, tough, training, troops, up, values, ve

# Tokenizing / vectorizing the whole dataset

* The datasets library allows us to efficiently map() a function across the whole dataset
* Can run in parallel

**Note**: confusingly, and unlike the Python`map` function, [`Dataset.map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) function _updates_ its argument dataset, keeping existing values. Here, the call adds the values returned by the function call (here `input_ids`) to each example while also keeping the original `text` and `label` values.


In [9]:
# Apply the tokenizer to the whole dataset using .map()
dset_tokenized = dset.map(vectorize_example)
pprint(dset_tokenized["train"][0])

Map: 100%|██████████| 25000/25000 [00:10<00:00, 2306.90 examples/s]

{'input_ids': [309,
               663,
               727,
               774,
               793,
               887,
               1115,
               1207,
               1980,
               2248,
               2390,
               2604,
               2625,
               3053,
               3696,
               4208,
               4293,
               4951,
               5265,
               5590,
               5617,
               5697,
               5835,
               6044,
               6118,
               6141,
               6357,
               6367,
               6370,
               6543,
               6663,
               6815,
               6878,
               6887,
               7120,
               7127,
               7346,
               7593,
               7801,
               8103,
               8241,
               8325,
               8350,
               8599,
               8649,
               9085,
               9391,
               9428

## Input encoding for MLP

* Our `input_ids` are an array containing the indices of the tokens found in the text
* This corresponds to the indices into the row of the embedding matrix in the model
* That seems to be exactly what we need!


# Batching and padding

* When working with neural networks, one rarely trains one example at a time
* Instead, processing always happens a batch at a time
* This has two important reasons:
  1. No batching is too slow (GPU parallelization cannot kick in across examples)
  2. The gradients are averaged across the whole batch and applied only once, i.e. batching acts as a regularizer and improves the stability of the training


# Padding and Collation (forming a batch)

## Padding:

* In order to build a batch as a 2D array of (example, seq), we need to fit together examples of different length
* Solution: pad the shorter examples with zeroes to the length of the longest example in the batch
* Make sure that zero is understood as padding value rather than a (hypothetical) feature with index 0
* This is best shown by example, it is in the end easier than it may sound

## Collation:

* Much like examples are dictionaries with the data, also batches are dictionaries with the data
* The only difference is that in a batch, all data tensors have one extra dimension, that's all there is to it

## Collator function:

* Padding and collation is taken care of by a single function in the HF libraries
* It receives a list of examples, and returns a ready batch
* The surrounding library code takes care of forming these lists
* Let's try to implement one below

In [10]:
# 1) I need to define it here, will explain below
# 2) I show here a very straightforward implementation of padding and collation
# 3) Normally, one would use transformers.DataCollatorWithPadding but that assumes
#    a particular tokenizer, to which it outsources much of the work, and we do not
#    have it
def collator(list_of_examples):
    #this is easy, labels are made into a single tensor
    batch={"labels":torch.tensor(list(ex["label"] for ex in list_of_examples))}
    #the worse bit is now to pad the examples, as they are of different length
    tensors=[]
    max_len=max(len(example["input_ids"]) for example in list_of_examples) #this is the longest example in the batch
    #everything needs to be padded to fit in length the longest example
    #(so we can build a single tensor out of it)
    for example in list_of_examples:
        ids=torch.tensor(example["input_ids"]) #pick the input ids
        # pad(what,(from_left, from_right)) <- this is how we call the stock pad function
        padded=torch.nn.functional.pad(ids,(0,max_len-ids.shape[0])) #pad by max - current length, pads with zero by default
        tensors.append(padded) #accumulated the padded ids
    batch["input_ids"]=torch.vstack(tensors) #now that we have all of them the same length, a simple vstack() stacks them up
    return batch #...and that's all there is to it



#Build a batch from 2 examples, with padding
batch=collator([dset_tokenized["train"][2],dset_tokenized["train"][7]])
print("Shape of labels:",batch["labels"].shape)
print("Shape of input_ids:",batch["input_ids"].shape)
pprint(batch["labels"])
pprint(batch["input_ids"])

Shape of labels: torch.Size([2])
Shape of input_ids: torch.Size([2, 153])
tensor([0, 1])
tensor([[  309,   417,   535,   663,   727,   774,   860,   887,   963,  1115,
          1207,  1228,  1337,  1377,  1495,  1517,  1702,  1756,  1764,  2237,
          2295,  2711,  3086,  3328,  3511,  3599,  3612,  3636,  4169,  4368,
          5089,  5372,  5429,  5434,  5490,  5552,  5798,  6044,  6288,  6308,
          6357,  6825,  6878,  6939,  7018,  7127,  7165,  7222,  7346,  7406,
          7472,  7697,  7777,  7932,  8117,  8128,  8290,  8322,  8484,  8783,
          8823,  8929,  9085,  9602,  9630,  9789,  9798,  9890, 10008, 10014,
         10263, 10350, 10398, 10475, 10508, 10513, 10558, 10576, 10638, 10829,
         10883, 10970, 11386, 11681, 11721, 11731, 11762, 11949, 12055, 12134,
         12202, 12239, 12363, 12437, 12445, 12539, 12618, 12712, 13209, 13285,
         13325, 13781, 14079, 14235, 14310, 14349, 14596, 14993, 15007, 15291,
         15502, 15603, 15776, 15778, 15835

# Build the MLP model

* Now that all of our data is in shape, we can build the model
* That is luckily quite easy in this case

The model class in its simplest form has `__init__()` which instantiates the layers and `forward()` which implements the actual computation. For more information on these, please see the [PyTorch turorial](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html).

In [11]:
import torch
import transformers

# A model wants a config, I can simply inherit from the base
# class for pretrained configs
class MLPConfig(transformers.PretrainedConfig):
    pass

# This is the model
class MLP(transformers.PreTrainedModel):

    config_class=MLPConfig

    # In the initialization method, one instantiates the layers
    # these will be, for the most part the trained parameters of the model
    def __init__(self,config):
        super().__init__(config)
        self.vocab_size=config.vocab_size #embedding matrix row count
        # Build and initialize embedding of vocab size +1 x hidden size (+1 because of the padding index 0!)
        self.embedding=torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0)
        # Normally you would not initialize these yourself, but I have my reasons here ;)
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001) #initialize the embeddings with small random values
        # Note! This function is relatively clever and keeps the embedding for 0, the padding, pure zeros
        # This takes care of the lower half of the network, now the upper half
        # Output layer: hidden size x output size
        self.output=torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels)
        # Now we have the parameters of the model


    # The computation of the model is put into the forward() function
    # it receives a batch of data and optionally the correct `labels`
    #
    # If given `labels` it returns (loss,output)
    # if not, then it returns (output,)
    def forward(self,input_ids,labels=None):
        #1) sum up the embeddings of the items
        embedded=self.embedding(input_ids) #(batch,ids)->(batch,ids,embedding_dim)
        # Since the Embedding keeps the first row of the matrix pure zeros, we don't need to worry about the padding
        # so next we sum the embeddings across the word dimension
        # (batch,ids,embedding_dim) -> (batch,embedding_dim)
        embedded_summed=torch.sum(embedded,dim=1)

        #2) apply non-linearity
        # (batch,embedding_dim) -> (batch,embedding_dim)
        projected=torch.tanh(embedded_summed) #Note how non-linearity is applied here and not when configuring the layer in __init__()

        #3) and now apply the upper, output layer of the network
        # (batch,embedding_dim) -> (batch, num_of_classes i.e. 2 in our case)
        logits=self.output(projected)

        # ...and that's all there is to it!

        #print("input_ids.shape",input_ids.shape)
        #print("embedded.shape",embedded.shape)
        #print("embedded_summed.shape",embedded_summed.shape)
        #print("projected.shape",projected.shape)
        #print("logits.shape",logits.shape)

        # If we have labels, we ought to calculate the loss
        if labels is not None:
            loss=torch.nn.CrossEntropyLoss() #This loss is meant for classification, so let's use it
            # You run it as loss(model_output,correct_labels)
            return (loss(logits,labels),logits)
        else:
            # No labels, so just return the logits
            return (logits,)



In [12]:
# Configure the model:
#   these parameters are used in the model's __init__()
mlp_config=MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=1,nlabels=2)

# And now we can instantiate it
mlp=MLP(mlp_config)

#we can make a little test with a fake batch formed by the two first example
fake_batch=collator([dset_tokenized["train"][0],dset_tokenized["train"][1]])
mlp(**fake_batch) #** expands input_ids and labels as parameters of the call

(tensor(0.6963, grad_fn=<NllLossBackward0>),
 tensor([[-0.6534, -0.7853],
         [-0.6594, -0.7869]], grad_fn=<AddmmBackward0>))

# Train the model

We will use the Hugging Face [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) class for training

* Loads of arguments that control the training
* Configurable metrics to evaluate performance
* Data collator builds the batches
* Early stopping callback stops when eval loss no longer improves
* Model load/save
* Excellent foundation for later deep learning course
  

First, let's create a [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/trainer#transformers.TrainingArguments) object to specify hyperparameters and various other settings for training.

Printing this simple dataclass object will show not only the values we set, but also the defaults for all other arguments. Don't worry if you don't understand what all of these do! Many are not relevant to us here, and you can find the details in [`Trainer` documentation](https://huggingface.co/docs/transformers/main_classes/trainer) if you are interested.

Next, let's create a metric for evaluating performance during and after training. We can use the convenience function [`load_metric`](https://huggingface.co/docs/datasets/about_metrics) to load one of many pre-made metrics and wrap this for use by the trainer.

As the task is simple binary classification and our data is even 50:50 balanced, we can comfortably use the basic `accuracy` metric, defined as the proportion of correctly predicted labels out of all labels.

In [13]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = np.argmax(outputs, axis=-1) #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)

We can then create the `Trainer` and train the model by invoking the [`Trainer.train`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.train) function.

In addition to the model, the settings passed in through the `TrainingArguments` object created above (`trainer_args`), the data, and the metric defined above, we create and pass the following to the `Trainer`:

* [data collator](https://huggingface.co/docs/transformers/main_classes/data_collator): groups input into batches
* [`EarlyStoppingCallback`](https://huggingface.co/docs/transformers/main_classes/callback#transformers.EarlyStoppingCallback): stops training when performance stops improving

In [14]:
# Argument gives the number of steps of patience before early stopping
# i.e. training is stopped when the evaluation loss fails to improve
# certain number of times
early_stopping = transformers.EarlyStoppingCallback(5)
results = []

for lr in [1e-4, 1e-3, 1e-2]:
    for bs in [10, 20, 40]:
        mlp = MLP(mlp_config)
        trainer_args = transformers.TrainingArguments(
            "mlp_checkpoints", #save checkpoints here
            evaluation_strategy="steps",
            logging_strategy="steps",
            eval_steps=500,
            logging_steps=500,
            learning_rate=lr, #learning rate of the gradient descent
            max_steps=20000,
            load_best_model_at_end=True,
            per_device_train_batch_size=bs
        )

        trainer = transformers.Trainer(
            model=mlp,
            args=trainer_args,
            train_dataset=dset_tokenized["train"],
            eval_dataset=dset_tokenized["test"].select(range(1000)), #make a smaller subset to evaluate on
            compute_metrics=compute_accuracy,
            data_collator=collator,
            callbacks=[early_stopping]
        )

        # FINALLY!
        trainer.train()

        eval_results = trainer.evaluate(dset_tokenized["test"])

        results.append((lr, bs, eval_results))
        print(f"For hyperparameters learning rate: {lr} and batch size: {bs} we get evaluation results: {eval_results}")

c:\Python310\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  2%|▎         | 500/20000 [00:02<01:17, 250.38it/s]

{'loss': 0.6583, 'grad_norm': 1.975420355796814, 'learning_rate': 9.75e-05, 'epoch': 0.2}


                                                    
  3%|▎         | 533/20000 [00:02<02:08, 151.06it/s]

{'eval_loss': 0.626163125038147, 'eval_accuracy': 0.755, 'eval_runtime': 0.2775, 'eval_samples_per_second': 3603.846, 'eval_steps_per_second': 450.481, 'epoch': 0.2}


  5%|▌         | 1000/20000 [00:04<01:17, 245.51it/s]

...

{'loss': 0.1392, 'grad_norm': 0.04071371629834175, 'learning_rate': 0.0085, 'epoch': 4.8}



 15%|█▌        | 3000/20000 [00:30<02:54, 97.55it/s]


{'eval_loss': 0.5314931273460388, 'eval_accuracy': 0.841, 'eval_runtime': 0.2918, 'eval_samples_per_second': 3427.187, 'eval_steps_per_second': 428.398, 'epoch': 4.8}
{'train_runtime': 30.7516, 'train_samples_per_second': 26014.942, 'train_steps_per_second': 650.374, 'train_loss': 0.24539825185139974, 'epoch': 4.8}


100%|██████████| 3125/3125 [00:05<00:00, 562.59it/s]

For hyperparameters learning rate: 0.01 and batch size: 40 we get evaluation results: {'eval_loss': 0.3659173548221588, 'eval_accuracy': 0.86, 'eval_runtime': 5.5597, 'eval_samples_per_second': 4496.67, 'eval_steps_per_second': 562.084, 'epoch': 4.8}


# Save the model for later use

* You can save it with `trainer.save_model()`
* You can load it with `MLP.from_pretrained()`


In [15]:
trainer.save_model("mlp-imdb")

# Check save/load

In [16]:
mlp2=MLP.from_pretrained("mlp-imdb")

In [17]:
trainer = transformers.Trainer(
    model=mlp2,
    args=trainer_args,
    train_dataset=dset_tokenized["train"],
    eval_dataset=dset_tokenized["test"],
    compute_metrics=compute_accuracy,
    data_collator=collator,
    callbacks=[early_stopping]
)

c:\Python310\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
eval_results = trainer.evaluate(dset_tokenized["test"])
print(eval_results)
print('Accuracy:', eval_results['eval_accuracy'])

100%|██████████| 3125/3125 [00:05<00:00, 523.46it/s]

{'eval_loss': 0.3659173548221588, 'eval_accuracy': 0.86, 'eval_runtime': 5.9729, 'eval_samples_per_second': 4185.545, 'eval_steps_per_second': 523.193}
Accuracy: 0.86


# Extra time left?

* Read through the TrainingArguments documentation, try to understand at least some parts of it https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
* Read through Torch tensor operations, try to understand at least some parts of it: https://pytorch.org/docs/stable/tensors.html
* Run the model with different parameters (hidden layer width, learning rate, etc), how much do the results change?


# What has the model learned?

* The embeddings should have some meaning to them
* Similar features should have similar embeddings

In [19]:
# Grab the embedding matrix out of the trained model
# and drop the first row (padding 0)
# then we can treat the embeddings as vectors
# and maybe compare them to each other
# ha ha this below took some googling
weights=mlp.embedding.weight.detach().cpu().numpy()
weights=weights[1:,:]

In [20]:
qry_idx=vectorizer.vocabulary_["lousy"] #embedding of "great"

#calculate the distance of the "lousy" embedding to all other embeddings
distance_to_qry=sklearn.metrics.pairwise.euclidean_distances(weights[qry_idx:qry_idx+1,:],weights)
nearest_neighbors=np.argsort(distance_to_qry) #indices of words nearest to "lousy"
for nearest in nearest_neighbors[0,:20]:
    print(idx2word[nearest])
# This works great!

lousy
apollo
whatsoever
crawl
curiosity
numbing
residence
naturally
glorified
minutes
tripe
77
sloppy
zeta
buck
annoyed
snakes
sounded
rainy
horribly


In [21]:
print(nearest_neighbors)

[[10693  1022 19543 ...  1459 13045  3473]]


* The embeddings indeed seem to reflect the task
* There is a meaning to them

# Feature weights

*   A typical "old-school" way to approach the classification would be a simple linear model, like LinearSVM
*   Under such model, each feature (word) would have a single one weight
*   And the classification would simply be based on the sum of these weights
*   In this context of this task, "positive" words would get a high weight, "negative" words would get a low weight
*   It is in fact quite easy to reconfigure the MLP model to work more or less like this and this effect can be replicated
*   I will leave that as an exercise for you



In [22]:
# weights now looks like this
print(weights)

[[ 0.27698907]
 [ 0.3083022 ]
 [-0.18851581]
 ...
 [-0.26665077]
 [ 0.20463663]
 [-0.03026071]]


In [23]:
# we dont want each weight to be an array item so lets reshape
weights = weights.reshape(1, -1)

In [24]:
# print the hundred most positive words
sorted_feature_importances=np.argsort(weights)
for most_positive in sorted_feature_importances[0,:100]:
    print(idx2word[most_positive])

coaster
perfect
awesome
excellent
wonderfully
gem
donald
favorites
today
enjoyed
amazing
maturity
jack
great
refreshing
notch
voight
lonely
adds
superb
wonderful
job
northam
forsythe
moving
naturalistic
fun
glamorous
ealing
solid
capote
tight
information
subtle
lang
piggy
raunchy
fortunately
favorite
marisa
swim
superlative
worlds
touching
enjoyable
testimony
cerebral
hatcher
appreciated
austen
fearful
best
timeless
stevenson
elam
unexpected
peterson
celebration
message
edie
spade
astonishing
succeeds
emma
soccer
scared
delightfully
gerard
initially
yelnats
1953
affection
screenings
favourite
passport
surprised
tenant
fantastic
kipling
comeuppance
everyman
boman
maintained
extravagant
remember
miyazaki
nagra
gundam
connolly
truth
position
email
april
coolest
engineer
chavez
recommended
pleasure
lucas
olympia


In [25]:
# print the hundred most negative words
for most_negative in reversed(sorted_feature_importances[0,-100:]):
    print(idx2word[most_negative])

worst
poorly
awful
disappointment
boring
waste
dreadful
dull
unconvincing
stinker
laughable
redeeming
distorted
fails
mess
disappointing
gag
bad
pointless
horrible
clone
mediocre
purposes
baldwin
ridiculously
unless
unintentional
weak
flop
sucks
incoherent
mst3k
stinks
turd
incomprehensible
atrocious
painful
fake
forgettable
poorest
letdown
trite
mediocrity
wasting
suffers
lifeless
blah
hoping
steaming
40s
whats
stilted
westerns
code
terrible
nauseous
secondly
strained
disgusting
unfunny
waster
clumsy
clunky
hodgepodge
miscast
worse
dramatically
stereotyped
pathetic
shame
spends
continuity
size
narcissistic
tedious
followed
choppy
unoriginal
contrived
proportions
asleep
prevent
sucked
brainless
synopsis
irritating
wig
flat
feminist
looks
mentally
sorely
sat
subjected
rubbish
backyard
jai
shirley
silvers
oh


In [33]:
for lr, bs, eval in results:
    print(f"Learning rate: {lr} and batch size: {bs}\nEvaluation accuracy: {eval['eval_accuracy']}\n")

Learning rate: 0.0001 and batch size: 10
Evaluation accuracy: 0.88672

Learning rate: 0.0001 and batch size: 20
Evaluation accuracy: 0.88752

Learning rate: 0.0001 and batch size: 40
Evaluation accuracy: 0.88876

Learning rate: 0.001 and batch size: 10
Evaluation accuracy: 0.87956

Learning rate: 0.001 and batch size: 20
Evaluation accuracy: 0.88544

Learning rate: 0.001 and batch size: 40
Evaluation accuracy: 0.88712

Learning rate: 0.01 and batch size: 10
Evaluation accuracy: 0.86356

Learning rate: 0.01 and batch size: 20
Evaluation accuracy: 0.84712

Learning rate: 0.01 and batch size: 40
Evaluation accuracy: 0.86



With a smaller learning rate, we get more accurate results. A larger batch size slightly increases accuracy but training also takes longer.